In [ ]:
import pyart
import wradlib as wrl
import pandas as pd
import tempfile
import os
import numpy as np
import matplotlib.pyplot as plt

import sys

import pytz
import datetime as dt

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#datestr = '2013/04/17'
#datestr = sys.argv[1]

path_radarfiles = '/lcrc/group/earthscience/icrisologo/diff_retrievals_processed/'+datestr

rr_retrieval_methods = ['rainrate','rainrate_from_kdp','rainrate_from_attenuation','rainrate_z_RT','rainrate_z_MP','rainrate_z_WC']

savepath = '/lcrc/group/earthscience/icrisologo/SAVEUR/gridded'



flist = os.listdir(path_radarfiles)
flist.sort()


In [ ]:
# create an empty dictionary
fname_dict = {}
for i in range(1,25,1):
    fname_dict[i] = []
# fill in dictionary
for i in range(len(flist)):
    #fname = keys[i].rsplit('/',1)[-1].strip('.nc')
    dtime_utc = dt.datetime.strptime(flist[i],'radar_KLOT_%Y%m%d_%H%M%S.nc')
    fname_dict[dtime_utc.hour+1].append(flist[i])

In [ ]:
# Get time intervals average every hour
timediff_ave = {}
for i in range(1,25,1):
    timediff_ave[i] = []
for hour in list(fname_dict):
    timelist = []
    timediff = []
    fnames_within_hour = fname_dict[hour]
    for fname in fnames_within_hour:
        dtime_utc = dt.datetime.strptime(fname,'radar_KLOT_%Y%m%d_%H%M%S.nc')
        timelist.append(dtime_utc)
    for tdiff in np.diff(timelist):
        timediff.append(np.diff(timelist)[0].seconds)
    timediff_ave[hour] = np.mean(timediff)

In [ ]:
# For every hour in the list, get the filenames associated with that hour and loop through the files. Read each file, 
# convert reflectivity to rain rate, and convert rain rate to rain amount. Add these rain amounts for 
# the hour and save it to file.

for hour in list(fname_dict):
    print('Hour: ',hour)
    interval = timediff_ave[hour]
    fnames_within_hour = fname_dict[hour]

    hour_accum = np.zeros((len(rr_retrieval_methods),300,300))
    for fname in fnames_within_hour:
        print(fname+':',end='')

        # get local time of radar
        fname_ = fname.rsplit('/',1)[-1].strip('.nc')
        dtime_utc = dt.datetime.strptime(fname_,'radar_KLOT_%Y%m%d_%H%M%S')
        dtime_utc = pytz.utc.localize(dtime_utc)

        # read to radar object using pyart
        radar = pyart.io.read_cfradial(os.path.join(path_radarfiles,fname))

#         # Mask out last 10 gates of each ray, this removes the "ring" around the radar.
#         radar.fields[method]['data'][:, -10:] = np.ma.masked

        gatefilter = pyart.filters.GateFilter(radar)
        gatefilter.exclude_transition()
        gatefilter.exclude_masked('reflectivity')

        # perform Cartesian mapping, limit to the reflectivity field.
        grid = pyart.map.grid_from_radars(
            [radar], gatefilters=[gatefilter],
            grid_shape=(1, 300, 300),
            grid_limits=((2000, 2000), (-123000.0, 123000.0), (-123000.0, 123000.0)),
            )

